In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd


model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def predict_pair(text1, text2):
    with torch.inference_mode():
        out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        return {v: proba[k] for k, v in model.config.id2label.items()}['entailment']

In [85]:
df = pd.read_csv('data/raw/paranmt_ru_leipzig.tsv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/paranmt_ru_leipzig.tsv'

In [4]:
df.head()

,idx,original,en,ru
0,276827,Жильцам многоквартирных домов часто приходится...,Residents of apartment buildings often have to...,Жителям многоквартирного дома часто приходится...
1,849426,"Так или иначе, желания наши все равно дадут о ...","In any case, our desires will still be felt.","В любом случае, наши желания все равно будут о..."
2,504500,"Негативные события станут позитивными тогда, к...",Negative events become positive when a person ...,"События становятся позитивными, если человек м..."
3,601055,Они уже стали чем-то вроде второй кожи в моем ...,They have already become something of a second...,"Они уже превратились для меня в нечто вроде ""в..."
4,980222,Юноши и девушки нового поколения имеют те же п...,Young men and women of the new generation have...,У юношей и девушек нового поколения одни и те ...


In [33]:
# import tqdm

# with open('leipzig_entailment.tsv', 'w', encoding='utf8') as f:
#     for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
#         ent1 = str(float(predict_pair(row['original'], row['ru'])))
#         ent2 = str(float(predict_pair(row['ru'], row['original'])))
#         f.write('\t'.join([str(row['idx']), row['original'], row['ru'], ent1, ent2]) + '\n')
        

100%|██████████| 1000000/1000000 [5:41:12<00:00, 48.85it/s] 


In [189]:
import pandas as pd

df = pd.read_csv('data/leipzig_entailment.tsv', sep='\t')

In [190]:
len(df)

1000000

In [191]:
df = df[(df.ent1 > 0.6) & (df.ent2 > 0.6)]

In [192]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()


df = df[~(df.text1.apply(lambda x: ad.is_latin(x)))]
df = df[~(df.text2.apply(lambda x: ad.is_latin(x)))]

In [194]:
from razdel import tokenize
from string import punctuation
from collections import Counter

c = Counter()

for i, row in df.iterrows():
    texts = (row['text1'], row['text2'])
    for t in texts:
        for char in t:
            c[char] += 1

filter_symbols = set([row[0] for row in c.most_common() if row[1] <= 459])
filter_symbols = set([ch for ch in filter_symbols if ch.strip() not in punctuation])
def filter_bad_char(s):
    s = set(s)
    for char in filter_symbols:
        if char in s:
            return True
    return False
        

In [195]:
df = df[~df.text1.apply(filter_bad_char)]
df = df[~df.text2.apply(filter_bad_char)]

In [198]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
from collections import Counter
from razdel import tokenize

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def match_ents(s1, s2):
    doc1, doc2 = Doc(s1), Doc(s2)
    
    doc1.segment(segmenter)
    doc2.segment(segmenter)
    
    doc1.tag_ner(ner_tagger)
    doc2.tag_ner(ner_tagger)
    
    if Counter([s.type for s in doc1.spans]) != Counter([s.type for s in doc2.spans]):
        return False
    
    numbers1 = set()
    numbers2 = set()
    
    for tok in doc1.tokens:
        if has_numbers(tok.text):
            numbers1.add(tok.text)
    
    for tok in doc2.tokens:
        if has_numbers(tok.text):
            numbers2.add(tok.text)
    
    if numbers1 != numbers2:
        return False
    
    return True
    

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def match_abbr(s1, s2):
    abbrs1 = set()
    abbrs2 = set()
    
    for tok in tokenize(s1):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs1.add(tok.text)
    for tok in tokenize(s2):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs2.add(tok.text)
    if abbrs1 == abbrs2:
        return True
    return False

def match_latin(s1, s2):
    lat1 = set()
    lat2 = set()
    
    for tok in tokenize(s1):
        if ad.islatin(tok.text):
            lat1.add(tok.text)
    for tok in tokenize(s2):
        if ad.islatin(tok.text):
            lat2.add(tok.text)

    if lat1 == lat2:
        return True
    return False

In [98]:
# cyrillic
# abbr
# numbers match
# ents match
# ты вы
# кавычки заменить на ёлочки все
# uniq ents

In [199]:
df = df[df.apply(lambda x: match_abbr(x['text1'], x['text2']), axis=1)]

In [203]:
import tqdm

ner_data = []
for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if match_ents(row['text1'], row['text2']):
        ner_data.append(row['id'])

100%|██████████| 523230/523230 [29:21<00:00, 296.96it/s]


In [206]:
df.columns

Index(['id', 'text1', 'text2', 'ent1', 'ent2'], dtype='object')

In [288]:
df = df[df.id.isin(ner_data)]

In [285]:
doc1, doc2 = Doc(s1), Doc(s2)

doc1.segment(segmenter)
doc2.segment(segmenter)

doc1.tag_ner(ner_tagger)
doc2.tag_ner(ner_tagger)



In [286]:
doc1.spans

[]

In [287]:
doc2.spans

[DocSpan(stop=10, type='LOC', text='Где-нибудь', tokens=[...])]

In [284]:
s1, s2 = k['text1'], k['text2']

In [344]:
k = df.sample(1).iloc[0]
print(k['text1'])
print(k['text2'])

В то же время встречаются родители, которые полностью одобряют увлечение детей, понимая, что просмотр аниме ничуть не опаснее комиксов или фильмов.
Вместе с тем, есть родители, полностью одобряющие увлечение своих детей, зная, что смотреть мультфильмы не более опасно, чем комедии или фильмы.


In [ ]:
df.to_csv('data/clean/leipzig_clean.tsv')